In [15]:
import os
import getpass
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# Load environment variables from .env file
load_dotenv()

# Get Google API key from .env
google_api_key = os.getenv("GOOGLE_API_KEY")

# Optional: Check if loaded
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file")

In [16]:
llm=ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    api_key=google_api_key,
    temperature=0.2)

In [17]:
def get_symptoms(state: dict)->dict:
    symptom=input("Welcome to Appolo Hospital, Please enter your symptoms: ")
    state["symptoms"]=symptom
    return state

In [18]:
def classify_symptom(state: dict) -> dict:
    prompt = (
        "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
        "-General\n -Emergency \n -mental health \n"
        f"Symptom : {state['symptom']} \n"
        "Respond only with one word : General, Emergency Or Mental Health"
        # "Example : input : I have fever, Output : General"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    category=response.content.strip()
    print(f"LLM classifies the symptom as: {category}")
    state["category"] = category
    return state
    

In [19]:
def symptom_router(state: dict) -> dict:
    cat = state["category"].lower()
    if "general" in cat:
        return "general"
    elif "emergency" in cat:
        return "emergency"
    elif "mental" in cat:
        return "mental_health"
    else:
        return "general"

In [20]:
def general_node(state: dict) -> dict:
    state["answer"] = f" {state['symptom']} : seems general : directing you to general ward for consulting a doctor"
    return state

def emergency_node(state: dict) -> dict:
    state["answer"] = f" {state['symptom']} : it is a Medical Emergency : seeking immediate help"
    return state

def mental_health_node(state: dict) -> dict:
    state["answer"] = f" {state['symptom']}: seems like a medical health issue; talk to our counsellor"
    return state